### Para quê paralelismo?

Ao longo dos últimos anos pudemos ver uma certa estagnação na evolução da velocidade dos núcleos dos processadores ainda que hoje existam bem mais núcleos disponíveis nos processadores de hoje em relação aos de anos atrás. Por isso mesmo buscar formas de explorar os demais núcleos se tornou um desafio para as linguagens de programação e aqui é que aparecem as coroutines, como subprocessos leves que têm a liberdade de operar em diferentes núcleos do processador de forma asíncrona.

Em Julia aproveitamos este recurso através de macros mas para termos a possibilidade de aproveitar por completo o processador, precisamos inicialmente iniciar os demais processos.

In [1]:
addprocs(1)

1-element Array{Int64,1}:
 2

In [2]:
procs()

2-element Array{Int64,1}:
 1
 2

Ao contrário de outras linguagens onde para iniciarmos uma thread precisaríamos criar uma classes e adicionar o método *run()*, em Julia basta apenas utilizarmos algumas macros

In [3]:
# criando um array de teste
m = SharedArray(Float64,100,100);
typeof(m)

SharedArray{Float64,2}

### @everywhare

Permite que uma função possa aproveitar todos os processos da execução atual

In [4]:
@everywhere function preenche(arr::SharedArray)
    for i in 1:length(arr)
        arr[i] = rand()
    end
end

In [5]:
@time preenche(m)

  0.050201 seconds (6.46 k allocations: 331.893 KB, 58.26% gc time)


### @parallel

Feito unicamente para o for, além de o tornar paralelo ainda aceita funções quase como um *reduce* aprimorado

In [6]:
@time @parallel (+) for i = 1:100000
    i
end

  1.532673 seconds (191.43 k allocations: 8.218 MB)


5000050000

In [8]:
@time @parallel for i = 1:length(m)
    m[i] = rand()
end

  0.014926 seconds (1.72 k allocations: 117.698 KB)


1-element Array{Any,1}:
 RemoteRef{Channel{Any}}(2,1,7)

### @async

In [9]:
task = @time @async @parallel for i = 1:length(m)
    m[i] = rand()
end

  0.005667 seconds (47 allocations: 3.940 KB)


Task (done) @0x00007fc9ad172270

In [ ]:
fib